# Getting all available Swiss supermarkets using the overpass Web API

## Libraries and settings

In [1]:
# Libraries
import os
import requests
import json
import folium
from pandas import json_normalize

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Show current working directory
print(os.getcwd())

/workspaces/data_analytics/Week_01


## Overpass turbo query to get all available supermarkets in Switzerland

In [2]:
# Overpass API URL
url = 'http://overpass-api.de/api/interpreter'

# Overpass turbo query
query = f"""
        [out:json];
        area["ISO3166-1"="CH"][admin_level=2];
        node ["shop"="supermarket"](area);
        out;"""

# Web API request
r = requests.get(url, params={'data': query})
data = r.json()['elements']

# Save data to file
with open('supermarkets.json', 'w') as json_file:
    json.dump(data, json_file)

# Store data in data frame
df = json_normalize(data)

# Number of rows and columns
print(df.shape)

# First rows
df.head()

(3437, 265)


,type,id,lat,lon,tags.brand,tags.brand:wikidata,tags.brand:wikipedia,tags.name,tags.opening_hours,tags.shop,...,tags.diet:organic,tags.diet:seafood,tags.brand:website,tags.recycling:low_energy_bulbs,tags.image,tags.opening_hours:cafe,tags.currency:XBT,tags.payment:lightning,tags.payment:lightning_contactless,tags.payment:onchain
0,node,33126515,47.155616,9.037915,Spar,Q610492,en:SPAR (retailer),Spar,Mo-Th 08:00-19:00; Fr 08:00-20:00; Sa 08:00-17:00,supermarket,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,36726161,47.226191,8.980329,Migros,Q680727,de:Migros,Migros,"Mo-Th 08:00-19:00, Fr 08:00-20:00, Sa 07:30-17...",supermarket,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,39768209,47.225154,8.969868,Coop,Q432564,NaN,Coop,NaN,supermarket,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,39947904,47.376732,8.542161,Coop,Q432564,de:Coop (Schweiz),Coop,Mo-Sa 06:00-22:00,supermarket,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node,48932835,47.375020,8.522895,Migros,Q680727,NaN,Migros,Mo-Sa 08:00-21:00; PH off,supermarket,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Plot supermarkets on map

In [3]:
# Subset of supermarkets by brand
locations = df[["lat", "lon", "tags.brand", "tags.shop"]].loc[df["tags.brand"].isin(['Migros', 'Coop'])]
print(locations.head(5))

# Create map
map = folium.Map(location=[locations.lat.mean(), 
                           locations.lon.mean()], 
                 zoom_start=8, 
                 control_scale=True)

# Add maker symbols
for index, location_info in locations.iterrows():
    folium.Marker([location_info["lat"], 
                   location_info["lon"]], 
                  popup=location_info["tags.brand"]).add_to(map)

# Plot map
map

         lat       lon tags.brand    tags.shop
1  47.226191  8.980329     Migros  supermarket
2  47.225154  8.969868       Coop  supermarket
3  47.376732  8.542161       Coop  supermarket
4  47.375020  8.522895     Migros  supermarket
7  47.491874  8.706448     Migros  supermarket


### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [4]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.8.0-1014-azure
Datetime: 2024-09-27 15:59:27
Python Version: 3.11.10
-----------------------------------
